In [ ]:
import cv2
import PIL
import io
import seaborn as sns
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [ ]:
picture_size = 48
main_path = "/kaggle/input/emotion-dataset/images/train"

In [ ]:
import os
emotion_categories = os.listdir(main_path)
emotion_categories

In [ ]:
# for categories paths
for subfolder in emotion_categories:
    category_path = os.path.join(main_path,subfolder)
    print(category_path)

In [ ]:
# Define the number of images to display from each category
num_images_per_category = 4

# Calculate total number of subplots needed
total_subplots = len(emotion_categories) * num_images_per_category

# Set up the figure and subplots
fig = plt.figure(figsize=(12, 12))

# Loop through each emotion category
for i, category in enumerate(emotion_categories):
    # Define the path to the specific emotion category folder
    category_folder = os.path.join(main_path, category)
    
    # Get the list of categories paths in the category folder
    image_files = os.listdir(category_folder)
    
    # Display a maximum of num_images_per_category images from this category
    for j in range(min(num_images_per_category, len(image_files))):
        # Construct the full path to the image file
        image_path = os.path.join(category_folder, image_files[j])
        
        # Load the image in grayscale
        image = mpimg.imread(image_path)
        
        # Create subplot
        ax = plt.subplot(len(emotion_categories), num_images_per_category, i * num_images_per_category + j + 1)
        
        # Display the grayscale image
        ax.imshow(image, cmap='gray')
        ax.set_title(category)
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
TRAIN_DIR = "/kaggle/input/emotion-dataset//images/train"
TEST_DIR = "/kaggle/input/emotion-dataset//images/validation"

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []

    try:
        for label in os.listdir(dir):
            for imagename in os.listdir(os.path.join(dir, label)):
                image_paths.append(os.path.join(dir, label, imagename))
                labels.append(label)
            print(label, "completed")
    except FileNotFoundError:
        print(f"Directory not found: {dir}")

    return image_paths, labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

In [ ]:
from PIL import Image
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = Image.open(image).convert('L')  # Load image in grayscale
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [ ]:
train_features = extract_features(train['image']) 

In [ ]:
test_features = extract_features(test['image'])

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation up to 20 degrees
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2, # Random vertical shift
    shear_range=0.2,        # Shear intensity
    zoom_range=0.2,         # Random zoom
    horizontal_flip=True,   # Random horizontal flip
    fill_mode='nearest'     # Fill mode for pixels outside the boundaries
)

# Example usage with a batch of images
# Assume x_train is your training data with shape (num_samples, height, width, channels)
# and y_train is your corresponding labels
datagen.fit(x_train)

# Create a generator that yields augmented batches of data
augmented_generator = datagen.flow(x_train, y_train, batch_size=128)

In [ ]:
# Callbacks

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=7,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=7,
    min_lr=1e-6,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

# Create the Sequential model
model = Sequential()

# Module 1
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(48, 48 , 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Module 2
model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Module 3
model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Flatten
model.add(Flatten())

# Dense layers
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(4, activation='softmax'))



model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
# Display the model summary
model.summary()

In [ ]:
history  = model.fit(augmented_generator, steps_per_epoch=int(len(x_train) / 128), epochs=200, validation_data=(x_test, y_test))

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from tensorflow.keras.models import model_from_json

json_file = open("/kaggle/working/emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("/kaggle/working/emotiondetector.h5")


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming you have your test data and labels ready
# x_test and y_true are the test data and true labels respectively

# Make predictions using the loaded model
y_pred_probabilities = model.predict(x_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)

y_test = np.argmax(y_test, axis=1)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()